In [39]:
import pandas as pd
from pycaret.time_series import *
import matplotlib.pyplot as plt
from fetch_data import fetch_data

In [53]:
df = fetch_data()

[*********************100%***********************]  30 of 30 completed


In [54]:
df.index = pd.to_datetime(df.index)
df = df.sort_index()

In [55]:
df.head()

,AAVE-USD,ADA-USD,ALGO-USD,ATOM-USD,AVAX-USD,BCH-USD,BNB-USD,BTC-USD,CHR-USD,DASH-USD,...,SOL-USD,THETA-USD,TRX-USD,UNI-USD,USDT-USD,VET-USD,WIF-USD,XLM-USD,XMR-USD,XRP-USD
Date,,,,,,,,,,,,,,,,,,,,,
2024-04-09,126.033913,0.591893,0.231800,10.835424,46.470196,672.589905,580.830322,69139.015625,0.398988,37.014671,...,172.413620,2.868446,0.121078,0.000208,0.999763,0.043554,3.588037,0.132163,133.521011,0.613817
2024-04-10,128.232162,0.586255,0.230523,10.810778,47.311695,629.482422,610.083923,70587.882812,0.394212,36.969437,...,173.437363,3.035096,0.119206,0.000212,1.000117,0.046501,3.624916,0.129653,133.582016,0.617561
2024-04-11,117.844589,0.586191,0.227998,10.755186,46.057598,614.317261,604.893799,70060.609375,0.392901,36.866966,...,172.576370,2.901548,0.120403,0.000210,1.000550,0.048192,3.459283,0.130300,133.369156,0.608880
2024-04-12,100.094933,0.503575,0.194067,9.387263,39.417744,534.766052,596.380127,67195.867188,0.334552,31.887470,...,153.635468,2.439062,0.114880,0.000202,1.000448,0.043033,2.672439,0.115299,122.996651,0.548414
2024-04-13,82.869781,0.447908,0.168352,8.092050,34.078411,481.483765,554.299805,63821.472656,0.326007,28.300234,...,139.341690,1.995765,0.110303,0.000192,1.001213,0.040311,2.345739,0.105686,115.678329,0.480819


In [76]:
btc = df.loc[:,'BTC-USD']
btc.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 365 entries, 2024-04-09 to 2025-04-08
Series name: BTC-USD
Non-Null Count  Dtype  
--------------  -----  
365 non-null    float64
dtypes: float64(1)
memory usage: 5.7 KB


In [87]:
# Set up PyCaret environment
s = setup(data=btc, 
          target = 'BTC-USD',
          transform_target='log',
          fh=30,  # Forecast horizon
          session_id=123,
          fold_strategy='expanding',
          seasonal_period='D',
          #fold_spans="adaptive",
          numeric_imputation_target = 'drift')

,Description,Value
0,session_id,123
1,Target,BTC-USD
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(365, 1)"
5,Transformed data shape,"(365, 1)"
6,Transformed train set shape,"(335, 1)"
7,Transformed test set shape,"(30, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [78]:
prophet = create_model('prophet')
#arima = create_model('arima')

In [79]:
tuned_prophet = tune_model(prophet)
#tuned_arima = tune_model(arima)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-12-09,4.9714,4.1599,17923.4752,20043.7950,0.1854,0.1662,-31.2155
1,2025-01-08,0.9293,0.8667,3386.0958,4188.1435,0.0346,0.0338,-0.1834
2,2025-02-07,2.3485,2.1208,8730.9984,10356.5841,0.0985,0.0919,-2.8370
Mean,NaT,2.7498,2.3825,10013.5232,11529.5075,0.1062,0.0973,-11.4120
SD,NaT,1.6744,1.3571,6003.7489,6525.9601,0.0618,0.0542,14.0450


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   15.2s finished


In [74]:
print(tuned_prophet)

ProphetPeriodPatched(changepoint_prior_scale=0.19615118354865432,
                     holidays_prior_scale=1.4906166728108328,
                     seasonality_prior_scale=0.11835503996887899)


In [80]:
# Predictions
final_prophet = finalize_model(tuned_prophet)
#future_dates = pd.date_range(df.index[-1], periods=30, freq='D')
prophet_predictions = predict_model(final_prophet)
#arima_predictions = predict_model(tuned_arima, fh=30)

In [82]:
#Plot prophet model forecast
plot_model(final_prophet, plot='forecast')



In [88]:
arima = create_model('arima')

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-12-09,1.3333,1.1959,4806.9143,5762.3896,0.0496,0.0483,-1.6626
1,2025-01-08,1.0532,0.8966,3837.7182,4332.6144,0.0383,0.0384,-0.2665
2,2025-02-07,1.3397,1.3018,4980.6148,6356.8002,0.0566,0.0543,-0.4456
Mean,NaT,1.2421,1.1314,4541.7491,5483.9347,0.0482,0.0470,-0.7916
SD,NaT,0.1336,0.1716,502.8503,849.5037,0.0075,0.0065,0.6203


In [89]:
tuned_arima = tune_model(arima)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-12-09,0.7755,0.7433,2795.7505,3581.4082,0.0281,0.0284,-0.0285
1,2025-01-08,1.5058,1.3368,5486.6403,6459.5071,0.0535,0.0555,-1.8151
2,2025-02-07,1.2567,1.3890,4671.8353,6782.8181,0.0539,0.0510,-0.6458
Mean,NaT,1.1793,1.1564,4318.0754,5607.9111,0.0452,0.0450,-0.8298
SD,NaT,0.3031,0.2929,1126.6711,1439.0200,0.0121,0.0119,0.7409


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:   19.2s finished


In [90]:
final_arima = finalize_model(tuned_arima)
#future_dates = pd.date_range(df.index[-1], periods=30, freq='D')
arima_predictions = predict_model(final_arima)

In [91]:
#Plot prophet model forecast
plot_model(final_arima, plot='forecast')

In [92]:
bnb = df.loc[:,'BNB-USD']
bnb.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 365 entries, 2024-04-09 to 2025-04-08
Series name: BNB-USD
Non-Null Count  Dtype  
--------------  -----  
365 non-null    float64
dtypes: float64(1)
memory usage: 5.7 KB


In [103]:
# Set up PyCaret environment
s = setup(data=bnb, 
          target = 'BNB-USD',
          #transform_target='log',
          fh=30,  # Forecast horizon
          session_id=123,
          fold_strategy='expanding',
          seasonal_period='D',
          #fold_spans="adaptive",
          numeric_imputation_target = 'drift')

,Description,Value
0,session_id,123
1,Target,BNB-USD
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(365, 1)"
5,Transformed data shape,"(365, 1)"
6,Transformed train set shape,"(335, 1)"
7,Transformed test set shape,"(30, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [104]:
arima = create_model('arima')

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-12-09,2.0868,1.7670,70.3869,76.9236,0.1014,0.0956,-15.6781
1,2025-01-08,1.2924,1.4175,42.5068,60.0780,0.0683,0.0637,-0.9224
2,2025-02-07,3.1391,2.5541,102.8631,108.4514,0.1619,0.1776,-9.0154
Mean,NaT,2.1727,1.9128,71.9189,81.8176,0.1105,0.1123,-8.5386
SD,NaT,0.7564,0.4753,24.6642,20.0493,0.0387,0.0479,6.0334


In [105]:
tuned_arima = tune_model(arima)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,14:35:09
Status,. . . . . . . . . . . . . . . . . .,Loading Dependencies
Estimator,. . . . . . . . . . . . . . . . . .,Compiling Library


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-12-09,0.6111,0.5861,20.6116,25.5135,0.0293,0.0297,-0.8347
1,2025-01-08,1.4359,1.5386,47.2284,65.2113,0.0758,0.0704,-1.2650
2,2025-02-07,1.0480,0.9909,34.3411,42.0739,0.0564,0.0546,-0.5074
Mean,NaT,1.0317,1.0385,34.0604,44.2662,0.0539,0.0516,-0.8690
SD,NaT,0.3369,0.3903,10.8681,16.2805,0.0191,0.0167,0.3102


Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:    7.9s finished


In [106]:
final_arima = finalize_model(tuned_arima)
#future_dates = pd.date_range(df.index[-1], periods=30, freq='D')
arima_predictions = predict_model(final_arima)

In [107]:
#Plot prophet model forecast
plot_model(final_arima, plot='forecast')